In [6]:
"""This script serves as a skeleton template for asynchronous AgentQL scripts."""

import logging
import asyncio

import agentql
from agentql.ext.playwright.async_api import Page
from playwright.async_api import async_playwright
import polars as pl

# Set up logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

URL = "https://leetcode.com/studyplan/leetcode-75/"


async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        try:
            # Create a new page in the browser and wrap it to get access to the AgentQL's querying API
            page = agentql.wrap(await browser.new_page())

            # Navigate to the desired URL
            await page.goto(URL)

            return await get_response(page)
        finally:
            await browser.close()


async def get_response(page: Page) -> dict:
    query = """
{
    problems[] {
        problem_name,
        section_title,
        difficulty,
        pill_items[]
    }
}
    """

    return await page.query_data(query)

# In Jupyter, use this to run the async code:
response = await main()



/Users/brandonbraner/projects/leetcode_scrape/.venv/lib/python3.12/site-packages/agentql/ext/playwright/sync_api/_utils_sync.py:46: RuntimeWarning: coroutine 'Page.evaluate' was never awaited
  page.evaluate(load_js("add_dom_change_listener"))
INFO:agentql:AgentQL query execution may take longer than expected, especially for complex queries and lengthy webpages. If you notice no activity in the logs, please be patient—the query is still in progress and has not frozen. The current timeout is set to 900 seconds, so you can expect a response within that timeframe. If a timeout error occurs, consider extending the timeout duration to give AgentQL backend more time to finish the work.
INFO:httpx:HTTP Request: POST https://api.agentql.com/api/v2/query-data "HTTP/1.1 200 OK"


In [7]:
# Create a dataframe from the response object problems
df = pl.DataFrame(response['problems'])

df.write_excel("leetcode-75.xlsx")
